# Classifying news articles with Naive Bayes

Once text data has been converted into numerical features using the natural language processing techniques discussed in the previous sections, text classification works just like any other classification task.

## Imports

In [1]:
using Pkg

In [2]:
using PyCall
using Conda

In [3]:
#Pkg.add("Glob")
#Pkg.add("TextAnalysis")
#Pkg.add("ScikitLearn")
#Pkg.add("StringEncodings")

In [4]:
using Glob
using TextAnalysis
using DataFrames
using ScikitLearn
using StringEncodings

In [5]:
using ScikitLearn: @sk_import

In [6]:
@sk_import feature_extraction.text: CountVectorizer
@sk_import naive_bayes: MultinomialNB
@sk_import metrics: accuracy_score
@sk_import metrics: confusion_matrix

PyObject <function confusion_matrix at 0x000000007F329430>

## News article classification

We start with an illustration of the Naive Bayes model for news article classification using the BBC articles that we read as before to obtain a DataFrame with 2,225 articles from 5 categories.

### Read BBC articles

In [7]:
files = Glob.glob("../data/bbc/bbc/**/*.txt")

2225-element Vector{String}:
 "..\\data\\bbc\\bbc\\business\\001.txt"
 "..\\data\\bbc\\bbc\\business\\002.txt"
 "..\\data\\bbc\\bbc\\business\\003.txt"
 "..\\data\\bbc\\bbc\\business\\004.txt"
 "..\\data\\bbc\\bbc\\business\\005.txt"
 "..\\data\\bbc\\bbc\\business\\006.txt"
 "..\\data\\bbc\\bbc\\business\\007.txt"
 "..\\data\\bbc\\bbc\\business\\008.txt"
 "..\\data\\bbc\\bbc\\business\\009.txt"
 "..\\data\\bbc\\bbc\\business\\010.txt"
 "..\\data\\bbc\\bbc\\business\\011.txt"
 "..\\data\\bbc\\bbc\\business\\012.txt"
 "..\\data\\bbc\\bbc\\business\\013.txt"
 ⋮
 "..\\data\\bbc\\bbc\\tech\\390.txt"
 "..\\data\\bbc\\bbc\\tech\\391.txt"
 "..\\data\\bbc\\bbc\\tech\\392.txt"
 "..\\data\\bbc\\bbc\\tech\\393.txt"
 "..\\data\\bbc\\bbc\\tech\\394.txt"
 "..\\data\\bbc\\bbc\\tech\\395.txt"
 "..\\data\\bbc\\bbc\\tech\\396.txt"
 "..\\data\\bbc\\bbc\\tech\\397.txt"
 "..\\data\\bbc\\bbc\\tech\\398.txt"
 "..\\data\\bbc\\bbc\\tech\\399.txt"
 "..\\data\\bbc\\bbc\\tech\\400.txt"
 "..\\data\\bbc\\bbc\\tech\\

In [8]:
doc_list = Any[]
for (i,file) in enumerate(files)
    f = open(file,"r")
    s = StringDecoder(f,"LATIN1", "UTF-8")
    topic = split(file, "\\")[end-1]
    lines = readlines(s)
    heading = strip(lines[1])
    body = strip(join([strip(line) for line in lines[2:end]]))
    push!(doc_list, [topic, heading, body])
    close(s)
    close(f)
end

In [9]:
docs = DataFrame(topic = [String(x[1]) for x in doc_list], heading = [String(x[2]) for x in doc_list], body = [String(x[3]) for x in doc_list])

,topic,heading,body
,String,String,String
1,business,Ad sales boost Time Warner profit,"Quarterly profits at US media giant TimeWarner jumped 76% to $1.13bn (Â£600m) for the three months to December, from $639m year-earlier.The firm, which is now one of the biggest investors in Google, benefited from sales of high-speed internet connections and higher advert sales. TimeWarner said fourth quarter sales rose 2% to $11.1bn from $10.9bn. Its profits were buoyed by one-off gains which offset a profit dip at Warner Bros, and less users for AOL.Time Warner said on Friday that it now owns 8% of search-engine Google. But its own internet business, AOL, had has mixed fortunes. It lost 464,000 subscribers in the fourth quarter profits were lower than in the preceding three quarters. However, the company said AOL's underlying profit before exceptional items rose 8% on the back of stronger internet advertising revenues. It hopes to increase subscribers by offering the online service free to TimeWarner internet customers and will try to sign up AOL's existing customers for high-speed broadband. TimeWarner also has to restate 2000 and 2003 results following a probe by the US Securities Exchange Commission (SEC), which is close to concluding.Time Warner's fourth quarter profits were slightly better than analysts' expectations. But its film division saw profits slump 27% to $284m, helped by box-office flops Alexander and Catwoman, a sharp contrast to year-earlier, when the third and final film in the Lord of the Rings trilogy boosted results. For the full-year, TimeWarner posted a profit of $3.36bn, up 27% from its 2003 performance, while revenues grew 6.4% to $42.09bn. ""Our financial performance was strong, meeting or exceeding all of our full-year objectives and greatly enhancing our flexibility,"" chairman and chief executive Richard Parsons said. For 2005, TimeWarner is projecting operating earnings growth of around 5%, and also expects higher revenue and wider profit margins.TimeWarner is to restate its accounts as part of efforts to resolve an inquiry into AOL by US market regulators. It has already offered to pay $300m to settle charges, in a deal that is under review by the SEC. The company said it was unable to estimate the amount it needed to set aside for legal reserves, which it previously set at $500m. It intends to adjust the way it accounts for a deal with German music publisher Bertelsmann's purchase of a stake in AOL Europe, which it had reported as advertising revenue. It will now book the sale of its stake in AOL Europe as a loss on the value of that stake."
2,business,Dollar gains on Greenspan speech,"The dollar has hit its highest level against the euro in almost three months after the Federal Reserve head said the US trade deficit is set to stabilise.And Alan Greenspan highlighted the US government's willingness to curb spending and rising household savings as factors which may help to reduce it. In late trading in New York, the dollar reached $1.2871 against the euro, from $1.2974 on Thursday. Market concerns about the deficit has hit the greenback in recent months. On Friday, Federal Reserve chairman Mr Greenspan's speech in London ahead of the meeting of G7 finance ministers sent the dollar higher after it had earlier tumbled on the back of worse-than-expected US jobs data. ""I think the chairman's taking a much more sanguine view on the current account deficit than he's taken for some time,"" said Robert Sinche, head of currency strategy at Bank of America in New York. ""He's taking a longer-term view, laying out a set of conditions under which the current account deficit can improve this year and next.""Worries about the deficit concerns about China do, however, remain. China's currency remains pegged to the dollar and the US currency's sharp falls in recent months have therefore made Chinese export prices highly competitive. But calls for a shift in Beijing's policy have fallen on deaf ears, despite recent comments

### Create stratified train-test split

We split the data into the default 75:25 train-test sets, ensuring that the test set classes closely mirror the train set:

In [10]:
using Random

function partitionTrainTest(data, at = 0.75)
    n = nrow(data)
    idx = shuffle(1:n)
    train_idx = view(idx, 1:floor(Int, at*n))
    test_idx = view(idx, (floor(Int, at*n)+1):n)
    data[train_idx,:], data[test_idx,:]
end

partitionTrainTest (generic function with 2 methods)

In [11]:
function get_factorized_topic(topic)
    if topic == "business"
        return 0
    elseif topic == "entertainment"
        return 1
    elseif topic == "politics"
        return 2
    elseif topic == "sport"
        return 3
    elseif topic == "tech"
        return 4
    end
end

get_factorized_topic (generic function with 1 method)

In [12]:
train, test = partitionTrainTest(docs, 0.75)

X_train = [String(substr_text) for substr_text in train.body]
X_test = [String(substr_text) for substr_text in test.body]

y_train = [get_factorized_topic(topic) for topic in train.topic]
y_test = [get_factorized_topic(topic) for topic in test.topic]

557-element Vector{Int64}:
 3
 3
 1
 3
 2
 2
 3
 1
 4
 4
 3
 1
 1
 ⋮
 0
 2
 2
 0
 0
 2
 1
 4
 3
 2
 3
 3

### Vectorize text data

We proceed to learn the vocabulary from the training set and transforming both dataset using the CountVectorizer with default settings to obtain almost 26,000 features:

In [13]:
vectorizer = CountVectorizer()
X_train_dtm = vectorizer.fit_transform(X_train)
X_test_dtm = vectorizer.transform(X_test)

PyObject <557x25917 sparse matrix of type '<class 'numpy.int64'>'
	with 109520 stored elements in Compressed Sparse Row format>

In [14]:
X_train_dtm.shape, X_test_dtm.shape

((1668, 25917), (557, 25917))

### Train Multi-class Naive Bayes model

In [15]:
classes = unique(y_train)
nbc = NaiveBayesClassifier(classes)

NaiveBayesClassifier{Int64}(String[], [3, 2, 0, 1, 4], Matrix{Int64}(undef, 0, 5))

In [16]:
for (text, class) in zip(X_train, y_train)
    TextAnalysis.fit!(nbc, text, class)
end

In [17]:
nb = MultinomialNB()
nb.fit(X_train_dtm, y_train)
y_pred_class = nb.predict(X_test_dtm)

557-element Vector{Int64}:
 3
 3
 1
 3
 2
 2
 3
 1
 4
 4
 3
 1
 1
 ⋮
 0
 2
 2
 0
 0
 2
 1
 4
 3
 2
 3
 3

### Evaluate Results

We evaluate the multiclass predictions using accuracy to find the default classifier achieved almost 98%:

#### Accuracy

In [18]:
accuracy_score(y_test, y_pred_class)

0.9766606822262118

#### Confusion matrix

In [19]:
DataFrame(confusion_matrix(y_true=y_test, y_pred=y_pred_class), :auto)

,x1,x2,x3,x4,x5
,Int64,Int64,Int64,Int64,Int64
1,141,0,4,0,4
2,1,97,1,0,1
3,0,0,89,0,0
4,1,1,0,131,0
5,0,0,0,0,86
